In [1]:
import streamlit as st
import pandas as pd
import ast
from ast import literal_eval
import chromadb
from chromadb.utils import embedding_functions
import openai
from pathlib import Path
import base64
import uuid
from openai import AzureOpenAI
from chromadb import Documents, EmbeddingFunction, Embeddings

In [2]:
def retreive_relevant_docs(chromdb_collection,prompt_embeddings):
    title_query_result = chromdb_collection.query(
        query_embeddings=prompt_embeddings,
        n_results=10,
        include=['metadatas','distances', 'documents']
    )
#     print(title_query_result)
    result = pd.DataFrame({
                'id':title_query_result['ids'][0], 
                'score':title_query_result['distances'][0],
                'docs':title_query_result['documents'][0],
                'metadata': title_query_result['metadatas'][0]
                })
    return result

In [3]:
def get_embeddings(input_text):
    client = AzureOpenAI(
    api_key = "3a3b78172a564e5187901b0b2a268100",
    api_version = "2023-05-15",
    azure_endpoint = "https://usammcomponentfactoryopenaiservicetmna.openai.azure.com/"
    )
    
    response = client.embeddings.create(
        input = input_text,
        model= "TextEmbeddingADA002"
    )
    
    return response.data[0].embedding

In [4]:
def get_response(prompt, system, temperature = 1.0):
    client = AzureOpenAI(
    api_key = "3a3b78172a564e5187901b0b2a268100",  
    api_version = "2023-05-15",
    azure_endpoint = "https://usammcomponentfactoryopenaiservicetmna.openai.azure.com/"
    )

    response = client.chat.completions.create(
        model="TurboCodeGPT35Turbo", # model = "deployment_name".
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": prompt}
        ],
        # max_tokens=3000
    )

    #print(response)
    # print(response.model_dump_json(indent=2))
    return response.choices[0].message.content

In [49]:
def create_context(df):
    context = ''
    # print(df['documents'])
    for i, row in df[['metadata', 'docs']].iterrows():
        
        context += f"Document {i+1} : \nProgram Name : {row['metadata']['program_name']}\nRule Sequence no. : {row['metadata']['Rule_Sequence_Number']}\nBusiness Rule : {row['docs']}\n\n"
        
#     print('context: ', context)
    return context

In [20]:
# # dtype = {'Code/Database': str, 'Functional Description':str,'English Translation':str}
# df = pd.read_excel('PASB400_annotations_refined.xlsx')
# df = df.drop_duplicates()

In [84]:
df = pd.read_csv('refined_embeddings.csv')
df['embeddings'] = df['embeddings'].apply(literal_eval)

In [85]:
df = df[~df['Functional_Description'].isna()]

In [86]:
df

,Annotation Type,Category,State,Source Code,Rule_Sequence_Number,Annotation_Description,Code/Database,Functional_Description,Modified By,English Translation,embeddings,program_name,id
0,RULE,Process Rule,IN_ANALYSIS,IF WS-BUS-ENT = ' ' ...,2.20,Check business entity of input file 01:\nThis ...,Code,"As a system user, I want the program to do the...",NaN,NaN,"[-0.03885219991207123, -0.0065142991952598095,...",PASB400,81029c08-8c35-4212-b08e-62008f62b166
1,RULE,Process Rule,IN_ANALYSIS,IF IN01-PROCESS-PERIOD = 'PPR0'...,2.30,Check process period of input file 01:\nThis c...,Code,"As a system user, I want the program to do the...",Subha Murugan,NaN,"[-0.040984105318784714, -0.0003497124998830259...",PASB400,f98a2a71-1367-41d0-b2f7-5caaa88f032d
2,RULE,Process Rule,IN_ANALYSIS,EXEC SQL ...,4.10,Fetch cursor C1-DENVCAL:\nThis code is fetchin...,Database,"Reads SSN1, period1, period2, period3, period4...",Subha Murugan,NaN,"[-0.016343001276254654, 0.0031679105013608932,...",PASB400,924c5659-f436-4cb6-8c69-0d1f4ddb3a15
3,RULE,Process Rule,IN_ANALYSIS,EVALUATE SQLCODE ...,4.20,Evaluate the sqlcode for fetch cursor C1-DENVC...,Code,"As a system user, I want the program to do the...",NaN,NaN,"[-0.01458784006536007, -0.007884727790951729, ...",PASB400,7a4142d2-1606-40e7-a386-d9875cf017d1
4,RULE,Process Rule,IN_ANALYSIS,IF WS-PROD-PERIOD = 'P...,4.30,Evaluate the sqlcode for fetch cursor C1-DENVC...,Code,"As a system user, I want the program to do the...",NaN,NaN,"[-0.019048471003770828, -0.00816467497497797, ...",PASB400,84236b67-2ccf-4f86-864a-a09ab8e3133e
5,RULE,Business Rule,IN_ANALYSIS,IF IN02-BUSINESS-ENTITY = WS-BUS-E...,5.30,Validate the business entity of PPO forecast s...,Code,"\nAs a system user, I want the program to do t...",Subha Murugan,NaN,"[-0.03342364355921745, -0.009586005471646786, ...",PASB400,c573e672-4d3b-4c7b-8f83-5f48fa8d6a5b
6,RULE,Business Rule,IN_ANALYSIS,IF WS-BUSENT-VALID ...,5.40,Validate duplicate record:\nIf the business en...,Code,"As a system user, I want the program to do the...",Subha Murugan,NaN,"[-0.031217889860272408, 0.0035749285016208887,...",PASB400,d25fcc09-7b0f-4cda-bd33-cfe753a20c99
7,RULE,Business Rule,IN_ANALYSIS,IF WS-BUSENT-VALID AND WS-NODUP-VA...,5.50,Validate SSN:\nIf the business entity is valid...,Code,"As a system user, I want the program to do the...",Subha Murugan,NaN,"[-0.029449740424752235, 0.01005618553608656, -...",PASB400,19a2677b-9582-4c66-bc2c-fe53e4c59d5a
8,RULE,Business Rule,IN_ANALYSIS,IF WS-BUSENT-VALID AND WS-REC-PPR...,5.60,Validate production year and month for record ...,Code,"As a system user, I want the program to do the...",Subha Murugan,NaN,"[-0.021784842014312744, 0.00676976190879941, -...",PASB400,d0a05804-4ff8-4aad-a685-94490bbb030b
9,RULE,Business Rule,IN_ANALYSIS,IF WS-BUSENT-VALID AND WS-REC-PPR...,5.70,Validate production year and month for record ...,Code,"As a system user, I want the program to do the...",Subha Murugan,NaN,"[-0.02116999961435795, 0.009458305314183235, -...",PASB400,f2be6627-1dab-4d3f-b36b-410d3ccb50ad


In [87]:
# df = df.rename(columns = {'Functional Description':'Functional_Description'})
# df = df.rename(columns = {'Rule Sequence Number':'Rule_Sequence_Number'})
# df = df.rename(columns = {'Annotation Description':'Annotation_Description'})

In [88]:
# embeddings = []
# for i in df['Functional_Description']:
#     embd = get_embeddings(i)
#     embeddings.append(embd)

In [89]:
# df['embeddings'] = embeddings

In [90]:
# df['program_name'] = 'PASB400'

In [91]:
# df['id'] = [str(uuid.uuid4()) for _ in range(len(df))]

In [92]:
# df['Functional_Description'] = df['Functional_Description'].astype(str)
# # df['embeddings'] = df['embeddings'].apply(literal_eval)

In [93]:
# df['embeddings']

In [94]:
# df.to_csv('refined_embeddings.csv', index=False)

In [95]:
result_df = df[['program_name','Source Code','Rule_Sequence_Number','Annotation_Description','Functional_Description', 'embeddings','id']]
result_df['id'] = result_df['id'].apply(str)

C:\Users\prenaik\AppData\Local\Temp\ipykernel_24684\2147554999.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['id'] = result_df['id'].apply(str)


In [97]:
result_df

,program_name,Source Code,Rule_Sequence_Number,Annotation_Description,Functional_Description,embeddings,id
0,PASB400,IF WS-BUS-ENT = ' ' ...,2.20,Check business entity of input file 01:\nThis ...,"As a system user, I want the program to do the...","[-0.03885219991207123, -0.0065142991952598095,...",81029c08-8c35-4212-b08e-62008f62b166
1,PASB400,IF IN01-PROCESS-PERIOD = 'PPR0'...,2.30,Check process period of input file 01:\nThis c...,"As a system user, I want the program to do the...","[-0.040984105318784714, -0.0003497124998830259...",f98a2a71-1367-41d0-b2f7-5caaa88f032d
2,PASB400,EXEC SQL ...,4.10,Fetch cursor C1-DENVCAL:\nThis code is fetchin...,"Reads SSN1, period1, period2, period3, period4...","[-0.016343001276254654, 0.0031679105013608932,...",924c5659-f436-4cb6-8c69-0d1f4ddb3a15
3,PASB400,EVALUATE SQLCODE ...,4.20,Evaluate the sqlcode for fetch cursor C1-DENVC...,"As a system user, I want the program to do the...","[-0.01458784006536007, -0.007884727790951729, ...",7a4142d2-1606-40e7-a386-d9875cf017d1
4,PASB400,IF WS-PROD-PERIOD = 'P...,4.30,Evaluate the sqlcode for fetch cursor C1-DENVC...,"As a system user, I want the program to do the...","[-0.019048471003770828, -0.00816467497497797, ...",84236b67-2ccf-4f86-864a-a09ab8e3133e
5,PASB400,IF IN02-BUSINESS-ENTITY = WS-BUS-E...,5.30,Validate the business entity of PPO forecast s...,"\nAs a system user, I want the program to do t...","[-0.03342364355921745, -0.009586005471646786, ...",c573e672-4d3b-4c7b-8f83-5f48fa8d6a5b
6,PASB400,IF WS-BUSENT-VALID ...,5.40,Validate duplicate record:\nIf the business en...,"As a system user, I want the program to do the...","[-0.031217889860272408, 0.0035749285016208887,...",d25fcc09-7b0f-4cda-bd33-cfe753a20c99
7,PASB400,IF WS-BUSENT-VALID AND WS-NODUP-VA...,5.50,Validate SSN:\nIf the business entity is valid...,"As a system user, I want the program to do the...","[-0.029449740424752235, 0.01005618553608656, -...",19a2677b-9582-4c66-bc2c-fe53e4c59d5a
8,PASB400,IF WS-BUSENT-VALID AND WS-REC-PPR...,5.60,Validate production year and month for record ...,"As a system user, I want the program to do the...","[-0.021784842014312744, 0.00676976190879941, -...",d0a05804-4ff8-4aad-a685-94490bbb030b
9,PASB400,IF WS-BUSENT-VALID AND WS-REC-PPR...,5.70,Validate production year and month for record ...,"As a system user, I want the program to do the...","[-0.02116999961435795, 0.009458305314183235, -...",f2be6627-1dab-4d3f-b36b-410d3ccb50ad


In [98]:
chroma_client = chromadb.Client()
chromdb_collection = chroma_client.get_or_create_collection(name='stories')

In [99]:
metadf = df[['program_name','Source Code','Rule_Sequence_Number','Annotation_Description']]
data = metadf.to_dict(orient='records')

In [100]:
chromdb_collection.add(
    ids=result_df.id.tolist(),
    embeddings=result_df.embeddings.tolist(),
    metadatas=data,
    documents=result_df.Functional_Description.tolist()
    )

Add of existing embedding ID: 81029c08-8c35-4212-b08e-62008f62b166
Add of existing embedding ID: f98a2a71-1367-41d0-b2f7-5caaa88f032d
Add of existing embedding ID: 924c5659-f436-4cb6-8c69-0d1f4ddb3a15
Add of existing embedding ID: 7a4142d2-1606-40e7-a386-d9875cf017d1
Add of existing embedding ID: 84236b67-2ccf-4f86-864a-a09ab8e3133e
Add of existing embedding ID: c573e672-4d3b-4c7b-8f83-5f48fa8d6a5b
Add of existing embedding ID: d25fcc09-7b0f-4cda-bd33-cfe753a20c99
Add of existing embedding ID: 19a2677b-9582-4c66-bc2c-fe53e4c59d5a
Add of existing embedding ID: d0a05804-4ff8-4aad-a685-94490bbb030b
Add of existing embedding ID: f2be6627-1dab-4d3f-b36b-410d3ccb50ad
Add of existing embedding ID: d780047e-6422-42cc-b11f-18dd671d2c86
Add of existing embedding ID: 04bdff14-ef07-4594-b7fe-be6d8fe16113
Add of existing embedding ID: 6a6e257f-4806-4d6a-916c-09d249bcb5bc
Add of existing embedding ID: f6889ece-2777-43f9-a575-c7722fc58dfd
Add of existing embedding ID: 302dc394-a1de-469e-90c5-cdd75a3f

In [114]:
try:
    # Get text input from the user
    user_query = None
    user_query_input = str(input("Enter some text:"))

    # Check if there is no input
    if user_query_input:
        user_query = user_query_input
        # print('inside 1111111111111')
    elif prompt1: 
        user_query = 'How is SSN validated ?'
        # print('inside 222222222222222')

    elif prompt2:
        user_query = "List all business rules related to production planning"
        # print('inside 33333333333333333')

    elif prompt3:
        user_query = "List all business rules related to production periods"
        # print('inside 444444444444444')

    if user_query:
#         print('*************user query:*************** ', user_query)
        title_query_result = retreive_relevant_docs(chromdb_collection, get_embeddings(user_query))
#         print('*************relevant documents**************')
#         print(title_query_result)
        # st.write(title_query_result['documents'])
        context = create_context(title_query_result)
        # print(context)

        # generate RAG responses by passing top relevant documents
        system_prompt = """
        You are an helpful assistant.
        """

        message_prompt = f"""
        
        User Query:
        ```
        {user_query}
        ```

        Contextual Documents:
        ```
        {context}
        ```
        
        You should adhere to display output in below format:
        
        'Similar Business Rules' : #Business rules functionally similar to the user query.
        'Citations' :
        1. Program Name : #Program Name from Contextual Documents.
        2. Rule Sequence no. : #Rule Sequence no. Contextual Documents.
        3. Similarity Score : #Similarity score for each business rule. 
        
        Do not display business rules that are not functionally similar to the user query.
        """

        response = get_response(message_prompt, system_prompt, temperature = 0.0)

        print(response)
        
except Exception as e:
    print(e)

Enter some text:list all business rules where duplicate records are handled
        Similar Business Rules:
        1. As a system user, I want the program to do the following validations:
    - If the business entity is valid
    - If the business entity, production year, production month, SSN, model year, PCA code, customer, specific position of the current record is the same as that of the previous record, then
      - Set 'ERDUP' as the error code and 'DUPLICATE RECORD EXISTS' as the error message.
      - Insert the current sales PPO forecast file data into the ITFFCTP table.
      - Write the output error message file.
    - Program Name: PASB400
    - Rule Sequence no.: 5.4
    - Similarity Score: 100%

     Citations: 
     1. Program Name : PASB400
     2. Rule Sequence no. : 5.4
     3. Similarity Score : 100%
     
     2. As a system user, I want the program to do the following validations:
    - If the current sales PPO forecast file record is not a duplicate
    - If the 